In [35]:
import sentencepiece as sp

In [36]:
N_TRAIN_SENTENCES = 10000
TRAINING_FILE = 'cnn/summary_bpe_train.txt'

# make training data

In [39]:
import glob
import random
import tqdm
files = glob.glob('cnn/summary/*')

In [40]:
with open(TRAINING_FILE, 'w') as f_train:
    for _ in tqdm.tqdm(range(N_TRAIN_SENTENCES)):
        sample_file = random.choice(files)
        with open(sample_file) as f_sample:
            sentence = random.choice(f_sample.read().split('\n'))
        f_train.write(f'{sentence}\n')

100%|██████████| 10000/10000 [00:01<00:00, 5519.36it/s]


# Fit Encoder

In [ ]:
import sentencepiece as spm
class BytePairEncoder:
    def __init__(self, vocab_size, model_name, train_files):
        self.vocab_size = vocab_size
        self.model_name = model_name
        self.model_file = f'{self.model_name}.model'
        self.vocab_file = f'{self.model_name}.vocab'
        self.processor = self._fit(train_files)

    def _fit(self, train_files):
        spm.SentencePieceTrainer.Train(' '.join((
            f'--input={train_files}',
            f'--model_prefix={self.model_name}',
            f'--vocab_size={self.vocab_size}'
            '--model_type=bpe',
        )))
        processor = spm.SentencePieceProcessor()
        processor.Load(self.model_file)
        return processor
        
    def encode(self, text):
        return np.array(self.processor.EncodeAsIds(text))
    
    def decode(self, ids):
        return self.processor.DecodeIds(ids)

In [ ]:
%time
BytePairEncoder(vocab_size=8000, model_name='cnn', train_files=TRAINING_FILE)